In [162]:
import numpy as np
import pandas as pd
import requests
import time
import hashlib
import base64
import uuid
import re
import schedule
from keras.models import load_model
import torch
import pandas as pd
import requests
import re
import os
import random
from tensorflow.keras.models import *
from keras import callbacks
from tensorflow.keras.layers import *
from keras.layers import *
from PIL import Image
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import torchvision
from torchvision import models
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import time
import torch.nn.functional as F
from sklearn.model_selection import KFold
import torch.optim as optim

In [72]:
def get_accessToken():
    base_key = 'y5lPd45nosR29tuu'
    pre_encrypt_key = 'qq1254771421'
    client_ID = "israel_h8um3n"
    user_name = "18378361441"
    u = "https://zhy.znznz.cn/Misc/Israel/login"

    t = int(time.time())
    md1 = hashlib.md5()
    md1.update(str.encode(str(t)))
    value = md1.hexdigest()
    random_key = value[-5:]
    key1 = base_key + random_key
    md2 = hashlib.md5()
    md2.update(str.encode(key1))
    encrypt_key = md2.hexdigest()
    encrypt_string = str(t) + pre_encrypt_key
    if len(encrypt_string) > len(encrypt_key):
        encrypt_key += encrypt_key
    r = ''
    for i in range(len(encrypt_string)):
        r += chr(ord(encrypt_string[i]) ^ ord(encrypt_key[i]))
    r = random_key + r
    test_result = base64.b64encode(r.encode('utf-8'))
    final_result = str(test_result, encoding='utf-8')

    password = final_result
    msg_ID = str(uuid.uuid1())
    data = {
        "username": user_name,
        "password": password,
        "clientId": client_ID,
        "msgId": msg_ID
    }
    response_login = requests.post(url=u, json=data)
    rlt = response_login.text.split(',')
    token = rlt[2].split(':')[2]
    return token[1:-1]

def segment_activities(df_s):
    activitiesSeq = []
    ponentialIndex = df_s.label.ne(df_s.label.shift())
    ii = np.where(ponentialIndex == True)[0]
    for i_s, end in enumerate(ii):
        if i_s > 0:
            df_stmp = df_s[ii[i_s - 1]:end]
            activitiesSeq.append(df_stmp)
    return activitiesSeq

def sequences_to_sentences(activity_sequences_s):
    sentences_s = []
    label_sentences_s = []
    for i_s in range(len(activity_sequences_s)):
        sentence = generate_sentence(activity_sequences_s[i_s])
        sentences_s.append(sentence)
        label_sentences_s.append(activity_sequences_s[i_s].label.values[0])
    return sentences_s, label_sentences_s

def generate_sentence(df2):
    sentence = ""
    sensors = df2.sensor.values
#    values = df2.status.values
    for i_s in range(len(sensors)):
#        val = values[i_s]
        if i_s == len(sensors) - 1:
            sentence += "{}".format(sensors[i_s])
        else:
            sentence += "{} ".format(sensors[i_s])
    return sentence

def sliding_window(sequence, win_size_s, step_s=1):
    try:
        iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    numOfChunks = int(((len(sequence) - win_size_s) / step_s) + 1)

    if win_size_s > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i_s in range(0, numOfChunks * step_s, step_s):
            yield sequence[i_s:i_s + win_size_s]

# 定义一个函数来填充1和0.5之间的值
def fill_between_values(col, df, col_name):
    first_one_index = col[col == 1].index.min()
    # 如果存在1的值，将其之前的所有值填充为0.5
    if first_one_index is not None:
        df.loc[:first_one_index - 1, col_name] = 0.5

    start_index = None
    previous_value = None
    for idx, value in col.items():
        if value == 1 or value == 0.5:
            if start_index is not None:
                if previous_value == 1:
                    df.loc[start_index:idx-1, col_name] = 1
                elif previous_value == 0.5:
                    df.loc[start_index:idx-1, col_name] = 0.5
                start_index = None
            previous_value = value
        elif start_index is None:
            start_index = idx

    return df[col_name]

def count_directories(path):
    dir_count = 0
    for root, dirs, files in os.walk(path):
        # 在第一层停止，不遍历子目录
        dir_count += len(dirs)
        break  # 只处理顶层目录，不进入子目录
    return dir_count

In [246]:
def get_new_data():
    ap = {
        "apicode": "5fb9d287",
        "begin_time": begin_time,
        "limit": 100000
    }
    raw_data = requests.post(url="https://zhy.znznz.cn/Misc/Israel/devlog", data=ap).text
    raw_data = re.split(r"[<>]", raw_data)[2]
    return raw_data


def process_data(raw_data):
    replace_dict = {
        'E1': '1',
        'E2': '2',
        'E3': '3',

        'S1': '11',
        'S2': '12',

        'L1': '21',
        'L2': '22',
        'L3': '23',
        'L4': '24',
        'L5': '25',
        'L6': '26',
        'L7': '27',

        'C1': '31',
        'C2': '32',
        'C3': '33',

        'D1': '41',
        'D2': '42',
        'D3': '43',
        'D4': '44',
        'D5': '45',
        'D6': '46',

        'K1': '51',

        'SM1':'61',
        'SM2':'62',

        'R1': '71',
        'R2': '72',

        
        '入口射灯':'100',
        '次卧主灯':'101',
        '次卧门口射灯':'102',
        '客厅主灯':'103',
        '餐厅灯':'104',
        '餐厅厨房灯':'105',
        '阳台灯':'106',
        '过道射灯':'107',
        '客厅调光射灯':'108',
        '主卧主灯':'110',
        '客厅射灯':'111',
        '餐厅射灯':'112',
        '主卧主灯':'113',
        '主卧射灯':'114',
        '主卧卫生间洗手灯':'115',
        '主卧卫生间主灯':'116',
        '次卧门口射灯':'117',
        '次卧主灯':'118',
        '卫生间洗手灯':'119',
        '卫生间灯':'120',
        '儿童房主灯':'121',
        '主卧睡眠':'122',
        '客厅窗帘':'123',

        '有人移动': '1',
        '2分钟无人移动': '0.5',

        '打开':'1',
        '关闭':'0.5'
    }

    #print(raw_data)

    data = [line for line in raw_data.split("\n") if line]

    # 对列表中的每个元素进行替换
    for i in range(len(data)):
        for key, value in replace_dict.items():
            data[i] = data[i].replace(key, value)

    # 将每行数据分割为单独的字段
    #split_data = [line.split() for line in data]
    df = [line.split() for line in data]

#    print(df[:10])
    return df


# 主函数

In [247]:
# 初始数据处理

raw_data = get_new_data()
df = process_data(raw_data)
begin_time = "2023-10-23 21:45:00"
img_index = 0
win_size = 8

with open('10_23.txt','a')as f:
    # 无限循环，以实时检查和处理新数据
    while True:
        new_raw_data = get_new_data()
        if new_raw_data != raw_data:
    #        print("New data received, processing...")
            raw_data = new_raw_data
            df = process_data(raw_data)
            df_new = preprocess(df, win_size)
            img_path = window2img(df_new, img_index, win_size)
            img_index += 1
            predicted_class = predict_image(img_path)
            f.write(f'{df_new}\n')  # \n 表示新行，这样每个 predicted_class 值会被写入到新的一行中
            f.write(f'{predicted_class}\n')  # \n 表示新行，这样每个 predicted_class 值会被写入到新的一行中
        time.sleep(0.1) 

[43.9  0.   0.   0.   0.   0.   0.   0. ]
Predicted Class: 14
[44.  43.9  0.   0.   0.   0.   0.   0. ]
Predicted Class: 14
[43.9 44.  43.9  0.   0.   0.   0.   0. ]
Predicted Class: 14
[44.  43.9 44.  43.9  0.   0.   0.   0. ]
Predicted Class: 14
[43.9 44.  43.9 44.  43.9  0.   0.   0. ]
Predicted Class: 14
[44.  43.9 44.  43.9 44.  43.9  0.   0. ]
Predicted Class: 14
[43.9 44.  43.9 44.  43.9 44.  43.9  0. ]
Predicted Class: 14
[44.  43.9 44.  43.9 44.  43.9 44.  43.9]
Predicted Class: 3
[43.9 44.  43.9 44.  43.9 44.  43.9 44. ]
Predicted Class: 3
[44.  43.9 44.  43.9 44.  43.9 44.  43.9]
Predicted Class: 3
[43.9 44.  43.9 44.  43.9 44.  43.9 44. ]
Predicted Class: 3
[44.  43.9 44.  43.9 44.  43.9 44.  43.9]
Predicted Class: 3
[43.9 44.  43.9 44.  43.9 44.  43.9 44. ]
Predicted Class: 3
[44.  43.9 44.  43.9 44.  43.9 44.  43.9]
Predicted Class: 3
[43.9 44.  43.9 44.  43.9 44.  43.9 44. ]
Predicted Class: 3
[44.  43.9 44.  43.9 44.  43.9 44.  43.9]
Predicted Class: 3
[43.9 44.  43.9 4

ProxyError: HTTPSConnectionPool(host='zhy.znznz.cn', port=443): Max retries exceeded with url: /Misc/Israel/devlog (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

# 预处理

In [243]:
def preprocess(df, win_size):
    # 转换为 DataFrame
    df = pd.DataFrame(df, columns=["date", "time", "sensor", "status"])

    df = df[df['sensor'] != 'sensor']
    df['sensor'] = pd.to_numeric(df['sensor'])
    df['status'] = df['status'].astype(float)
    #print(df[:10])

    # 定义特殊的sensor值
    special_sensors = list(range(100, 120))  # 从100到119的列表
    for sensor in special_sensors:
            # 删除包含特殊sensor值的行
        df = df[df["sensor"] != sensor]

    # 获取 'status' 和 'sensor' 列
    # 删除包含错误值的行
    df = df[df['sensor'] != 'sensor']
    df['sensor'] = pd.to_numeric(df['sensor'])
    sensor_column = df['sensor'].astype(float)
    status_column = df['status'].astype(float)

    # 将 'status' 列中所有1变成0, 所有0.5变成-0.1
    status_column[status_column == 1] = 0
    status_column[status_column == 0.5] = -0.1

    # 将处理后的 'status' 列和 'sensor' 列相加，并将结果赋值给 'sensor' 列
    df['sensor'] = status_column + sensor_column

    # 输出结果
#    print(df[:10])

#    win_size = 8
    # 提取 'sensor' 列的前x行
    sensor_values = df['sensor'].head(win_size).values

    # 如果实际的行数少于x，你可能想要填充数组以保持固定的大小
    if len(sensor_values) < win_size:
        padding = win_size - len(sensor_values)
        sensor_values = np.pad(sensor_values, (0, padding), 'constant', constant_values=(0, 0))
    print(sensor_values)
    sensor_values = sensor_values[::-1]
    return sensor_values

# 转为图像

In [235]:
def window2img(df,img_index, win_size):
    label_dir = f"/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset/1013/plot_train_state_6"

    for i in range(win_size):
        zero = np.zeros((100,100))
        for j in range(win_size):
            index = 99 - df[j]
            if index != 0:
                if index - index.astype(int) != 0:
                    index = index.astype(int)
                    zero[index][2*j:(2*j+1)] = 0.5
                elif index - index.astype(int) == 0:
                    index = index.astype(int)
                    zero[index][2*j:(2*j+1)] = 1
                else:
                    zero[index][2*j:(2*j+1)] = 0

    ##############状态填充    
        for row in range(100):
            # 找到当前纵坐标下所有值不为0的点的横坐标
            cols_with_value = [col for col in range(100) if zero[row][col] > 0]

            if len(cols_with_value) > 0:
                # 如果第一个值是0.5，从y=0向这一点连线
                if zero[row][cols_with_value[0]] == 0.5:
                    zero[row][0:cols_with_value[0]] = 1

                # 对相邻的on和off之间进行填充
                for k in range(0, len(cols_with_value) - 1):
                    # If the current point is 'on' and the next point is 'off', fill between them
                    if zero[row][cols_with_value[k]] == 1 and zero[row][cols_with_value[k + 1]] == 0.5:
                        zero[row][cols_with_value[k]:cols_with_value[k + 1]] = 1

                # 如果最后一个值为1，从这点向y=not_nan_count*2 - 1连线
                if zero[row][cols_with_value[-1]] == 1:
                    zero[row][cols_with_value[-1]:2*win_size-1] = 1

    # 找到当前纵坐标下所有值为1的点的横坐标
        for row in range(100):
            cols_with_value = [col for col in range(100) if zero[row][col] == 1]

            # 将除了值为1的点外的其他点设置为0
            zero[row, :] = 0
            for col in cols_with_value:
                zero[row, col] = 1
            ###########

        # 将zero数组的值乘以255
        data = zero * 255
        # 将data数组的数据类型转换为'uint8'
        data = data.astype('uint8')
        # 确保保存图像的目录存在，如果不存在就创建
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)

        # 生成图片文件的完整路径，这里使用一个计数器以防止覆盖同一标签下的不同图片
        img_path = os.path.join(label_dir, f"{img_index}.png")

        # 将图像数据保存为图片
        img = Image.fromarray(data)
        img.save(img_path)

        # 重置zero数组为224x224的零数组，为下一次迭代做准备
        zero = np.zeros((100, 100))
        
        return img_path

# 实时训练

In [239]:
net = models.resnet34()
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
device = torch.device("mps")

#4是事件，5是状态
wts_path = r'/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset/1013/plot_train_state_5/best_model.pth'
loaded_model_wts = torch.load(wts_path, map_location=device)
net.load_state_dict(loaded_model_wts)
net.eval()  # 将模型设置为评估模式

net.to(device)  # 将模型移动到CUDA设备上

def predict_image(img_path):
    # 加载图像
    img = Image.open(img_path).convert('L')  # 确保图像是灰度格式

    # 定义转换
    transform = transforms.Compose([
        transforms.Resize((100, 100)),  # 将图像大小调整为适合网络的大小
        transforms.ToTensor(),  # 将图像转换为Tensor
    ])

    # 应用转换
    img = transform(img).unsqueeze(0)  # 添加批处理维度
    # 将图像数据移动到模型所在的设备
    img = img.to(device)

    # 进行预测
    with torch.no_grad():
        output = net(img)
        _, predicted = output.max(1)  # 获取预测类别
    
    print(f'Predicted Class: {predicted.item()}')  # 打印预测的类别
    return predicted.item()

# 灯控制

In [249]:
def get_accessToken():
    base_key = 'y5lPd45nosR29tuu'
    pre_encrypt_key = 'qq1254771421'
    client_ID = "israel_h8um3n"
    user_name = "18378361441"
    u = "https://zhy.znznz.cn/Misc/Israel/login"

    t = int(time.time())
    md1 = hashlib.md5()
    md1.update(str.encode(str(t)))
    value = md1.hexdigest()
    random_key = value[-5:]
    key1 = base_key + random_key
    md2 = hashlib.md5()
    md2.update(str.encode(key1))
    encrypt_key = md2.hexdigest()
    encrypt_string = str(t) + pre_encrypt_key
    if len(encrypt_string) > len(encrypt_key):
        encrypt_key += encrypt_key
    r = ''
    for i in range(len(encrypt_string)):
        r += chr(ord(encrypt_string[i]) ^ ord(encrypt_key[i]))
    r = random_key + r
    test_result = base64.b64encode(r.encode('utf-8'))
    final_result = str(test_result, encoding='utf-8')

    password = final_result
    msg_ID = str(uuid.uuid1())
    data = {
        "username": user_name,
        "password": password,
        "clientId": client_ID,
        "msgId": msg_ID
    }
    response_login = requests.post(url=u, json=data)
    rlt = response_login.text.split(',')
    token = rlt[2].split(':')[2]
    return token[1:-1]


def get_device_id(accesstoken):
    u = "https://zhy.znznz.cn/Misc/Israel/device_list"
    msgId = str(uuid.uuid1())
    data = {
        "accessToken": accesstoken,
        "msgId": msgId,
    }
    response = requests.post(url=u, json=data).text
    return response

def send_on_off_information(on_device, off_device):
    if len(on_device) == 0:
        print('All switch closed')
    else:
        print('Open ', on_device[:, 0])

    if len(off_device) == 0:
        print('All switch opened')
    else:
        print('Close ', off_device[:, 0])


def control_light(on_device, off_device):
    msgId = str(uuid.uuid1())
    url = "https://zhy.znznz.cn/Misc/Israel/device_control"
    control_data = {
        "accessToken": accesstoken,
        "msgId": msgId,
#        "deviceId": '134929',
        "Attr": {
            "switch": 0
        }
    }
    # response = requests.post(url=url, json=control_data).text
    # print('Control result :', response)


def control_light_information(light_on, light_off):
    box = control_device_index
    on_device = []
    off_device = []
    for i in range(len(light_on)):
        on_device.append(box[light_on[i], :])
    on_device = np.array(on_device)
    for i in range(len(light_off)):
        off_device.append(box[light_off[i], :])
    off_device = np.array(off_device)
    send_on_off_information(on_device, off_device)
    # control_light(on_device, off_device)

    
def lights_replace(label):
    lights_id = {
        '入口射灯':'100',
        '次卧主灯':'101',
        '次卧门口射灯':'102',
        '客厅主灯':'103',
        '餐厅灯':'104',
        '餐厅厨房灯':'105',
        '阳台灯':'106',
        '过道射灯':'107',
        '客厅调光射灯':'108',
        '主卧主灯':'110',
        '客厅射灯':'111',
        '餐厅射灯':'112',
        '主卧主灯':'113',
        '主卧射灯':'114',
        '主卧卫生间洗手灯':'115',
        '主卧卫生间主灯':'116',
        '次卧门口射灯':'117',
        '次卧主灯':'118',
        '卫生间洗手灯':'119',
        '卫生间灯':'120',
        '儿童房主灯':'121',
        '主卧睡眠':'122',
        '客厅窗帘':'123',
    }
    # 对列表中的每个元素进行替换
    for i in range(len(label)):
        for key, value in replace_dict.items():
            label[i] = label[i].replace(key, value)

mapping = {0: '[0,0,0,0,0,0,0]', 1: '[0,0,0,0,0,0,1]', 2: '[0,0,0,0,0,1,0]', 3: '[0,0,0,0,1,0,0]', 4: '[0,0,0,0,1,0,1]', 5: '[0,0,0,0,1,1,0]', 6: '[0,0,0,1,0,0,0]', 7: '[0,0,0,1,1,0,0]', 8: '[0,1,1,0,0,0,0]', 9: '[0,1,1,1,0,0,0]', 10: '[1,0,0,0,0,0,0]', 11: '[1,0,0,1,0,0,0]', 12: '[1,0,1,0,0,0,0]', 13: '[1,0,1,1,0,0,0]', 14: '[1,1,1,0,0,0,0]', 15: '[1,1,1,1,0,0,0]'}

In [250]:
predicted.item


accesstoken = get_accessToken()
device_id = get_device_id(accesstoken)
print(device_id)

#control_device_index = index_to_list(switch_index, device_id)

{"code":0,"msg":"ok","data":{"Devices":[{"deviceId":"130843","deviceName":"客餐厅藏光灯","deviceType":"Switch","online":1},{"deviceId":"130844","deviceName":"客厅射灯","deviceType":"Switch","online":1},{"deviceId":"130845","deviceName":"过道射灯","deviceType":"Switch","online":1},{"deviceId":"130849","deviceName":"卫生间灯","deviceType":"Switch","online":1},{"deviceId":"130850","deviceName":"卫生间洗手灯","deviceType":"Switch","online":1},{"deviceId":"130851","deviceName":"卫生间排气","deviceType":"Switch","online":1},{"deviceId":"130857","deviceName":"餐厅射灯","deviceType":"Switch","online":1},{"deviceId":"130858","deviceName":"餐厅灯","deviceType":"Switch","online":1},{"deviceId":"130859","deviceName":"餐厅厨房灯","deviceType":"Switch","online":1},{"deviceId":"132616","deviceName":"客厅纱帘","deviceType":"Curtain","online":1},{"deviceId":"132617","deviceName":"客厅窗帘","deviceType":"Curtain","online":1},{"deviceId":"132618","deviceName":"过道客厅关","deviceType":"Switch","online":1},{"deviceId":"132619","deviceName":"过道窗帘","deviceType